In [ ]:
import sys
import math
import random
from collections import defaultdict


def main():
    n_strings, k_size = map(int, input().split())
    alphabet_str = list(input())
    raw_strings = [input() for _ in range(n_strings)]
    char_to_idx = {c: i for i, c in enumerate(alphabet_str)} 
    seq_data = []
    for s in raw_strings:
        indices = [char_to_idx[c] for c in s]
        start_char = indices[0]
        
        transitions = []
        counts = defaultdict(int)
        
        for i in range(len(indices) - 1):
            u = indices[i]
            v = indices[i+1]
            counts[(u, v)] += 1
        
        last_u = indices[-1]
        last_key = (last_u, k_size - 1)
        counts[last_key] += 1
        
        for (u, v), count in counts.items():
            transitions.append((u, v, count))
            
        seq_data.append({
            'start': start_char,
            'trans': transitions
        })

    num_restarts = 15
    max_iters = 25
    smoothing = 0.1
    
    best_total_ll = -float('inf')
    best_labels = [0] * n_strings

    for _ in range(num_restarts):
        current_labels = [random.randint(0, 1) for _ in range(n_strings)]
        current_labels[0] = 0
        current_labels[1] = 1
        
        iter_total_ll = 0
        
        for _ in range(max_iters):
            counts_start = [[smoothing] * (k_size - 1) for _ in range(2)]
            counts_trans = [[[smoothing] * k_size for _ in range(k_size - 1)] for _ in range(2)]
            
            for i in range(n_strings):
                lab = current_labels[i]
                item = seq_data[i]
                
                counts_start[lab][item['start']] += 1
                for u, v, c in item['trans']:
                    counts_trans[lab][u][v] += c

            log_start = [[0.0] * (k_size - 1) for _ in range(2)]
            log_trans = [[[0.0] * k_size for _ in range(k_size - 1)] for _ in range(2)]
            
            for clust in range(2):
                sum_s = sum(counts_start[clust])
                log_sum_s = math.log(sum_s)
                for i in range(k_size - 1):
                    log_start[clust][i] = math.log(counts_start[clust][i]) - log_sum_s
                
                for r in range(k_size - 1):
                    sum_r = sum(counts_trans[clust][r])
                    log_sum_r = math.log(sum_r)
                    for c in range(k_size):
                        log_trans[clust][r][c] = math.log(counts_trans[clust][r][c]) - log_sum_r

            new_labels = [0] * n_strings
            iter_total_ll = 0
            changed = False
            
            for i in range(n_strings):
                item = seq_data[i]
                s_char = item['start']
                trans_list = item['trans']
                
                ll_0 = log_start[0][s_char]
                for u, v, c in trans_list:
                    ll_0 += c * log_trans[0][u][v]
                    
                ll_1 = log_start[1][s_char]
                for u, v, c in trans_list:
                    ll_1 += c * log_trans[1][u][v]
                
                if ll_1 > ll_0:
                    new_labels[i] = 1
                    iter_total_ll += ll_1
                else:
                    new_labels[i] = 0
                    iter_total_ll += ll_0
                    
                if new_labels[i] != current_labels[i]:
                    changed = True
            
            current_labels = new_labels
            if not changed:
                break
        
        if iter_total_ll > best_total_ll:
            best_total_ll = iter_total_ll
            best_labels = list(current_labels)

    print('\n'.join(map(str, best_labels)))


if __name__ == '__main__':
    main()

In [ ]:
import math
import random
import sys

def main():
    n, k = map(int, input().split())
    alphabet_str = input().strip()
    char_to_idx = {c: i for i, c in enumerate(alphabet_str)}
    strings = []
    for _ in range(n):
        encoded = [char_to_idx[c] for c in input().strip()]
        strings.append(encoded)

    num_clusters = 2
    iterations = 40
    eps = 1e-10

    gamma = [[0.0] * num_clusters for _ in range(n)]
    for i in range(n):
        p0 = random.uniform(0.4, 0.6)
        gamma[i][0] = p0
        gamma[i][1] = 1.0 - p0

    start_probs = [[0.0] * (k - 1) for _ in range(num_clusters)]
    trans_probs = [[[0.0] * k for _ in range(k - 1)] for _ in range(num_clusters)]

    for _ in range(iterations):
        start_counts = [[eps] * (k - 1) for _ in range(num_clusters)]
        trans_counts = [[[eps] * k for _ in range(k - 1)] for _ in range(num_clusters)]
        cluster_weights = [eps] * num_clusters

        for i in range(n):
            s = strings[i]
            for c in range(num_clusters):
                w = gamma[i][c]
                cluster_weights[c] += w
                start_counts[c][s[0]] += w
                
                for j in range(len(s) - 1):
                    u, v = s[j], s[j+1]
                    trans_counts[c][u][v] += w
                
                trans_counts[c][s[-1]][k - 1] += w

        for c in range(num_clusters):
            sum_start = sum(start_counts[c])
            for char_idx in range(k - 1):
                start_probs[c][char_idx] = start_counts[c][char_idx] / sum_start
            
            for u in range(k - 1):
                sum_trans = sum(trans_counts[c][u])
                for v in range(k):
                    trans_probs[c][u][v] = trans_counts[c][u][v] / sum_trans
        
        for i in range(n):
            s = strings[i]
            log_likelihoods = [0.0] * num_clusters
            
            for c in range(num_clusters):
                likelihood = math.log(start_probs[c][s[0]])
                for j in range(len(s) - 1):
                    log_likelihoods[c] += math.log(trans_probs[c][s[j]][s[j+1]])
                log_likelihoods[c] += math.log(trans_probs[c][s[-1]][k - 1]) + math.log(cluster_weights[c] / n)
            
            probs = [math.exp(likelihood - max(log_likelihoods)) for likelihood in log_likelihoods]
            sum_probs = sum(probs)
            
            for c in range(num_clusters):
                gamma[i][c] = probs[c] / sum_probs

    results = []
    for i in range(n):
        if gamma[i][1] > gamma[i][0]:
            results.append(1)
        else:
            results.append(0)

    print('\n'.join(map(str, results)))

if __name__ == "__main__":
    main()